# 스트리밍하는 방법


!!! 정보 "전제 조건"

    이 가이드는 다음 사항에 대한 친숙성을 가정합니다:
    
    - [스트리밍](../../concepts/streaming/)
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)

스트리밍은 LLM(대형 언어 모델) 기반 애플리케이션의 반응성을 개선하는 데 필수적입니다. 출력 결과를 점진적으로 표시함으로써, 완전한 응답이 준비되기 이전에도 스트리밍은 사용자 경험(UX)을 크게 향상시킵니다. 특히 LLM의 지연 문제를 다루는 데 유용합니다.

LangGraph는 스트리밍을 일급 지원합니다. 그래프 실행에서 출력을 스트리밍하는 여러 가지 방법이 있습니다:

- `"values"`: 각 단계 후 상태의 모든 값을 방출합니다.
- `"updates"`: 각 단계 후 노드가 반환하는 노드 이름과 업데이트만 방출합니다.
    같은 단계에서 여러 업데이트가 이루어진 경우(예: 여러 노드 실행) 해당 업데이트는 별도로 방출됩니다.
- `"custom"`: `StreamWriter`를 사용하여 노드 내부에서 사용자 정의 데이터를 방출합니다.
- [`"messages"`](../streaming-tokens): 노드 내부의 LLM 호출에 대한 메타데이터와 함께 LLM 메시지를 토큰 단위로 방출합니다.
- `"debug"`: 각 단계에 대해 가능한 많은 정보를 포함한 디버그 이벤트를 방출합니다.

`graph.stream(..., stream_mode=<stream_mode>)` 메서드를 사용하여 그래프에서 출력을 스트리밍할 수 있습니다. 예를 들어:

=== "동기"

    ```python
    for chunk in graph.stream(inputs, stream_mode="updates"):
        print(chunk)
    ```

=== "비동기"

    ```python
    async for chunk in graph.astream(inputs, stream_mode="updates"):
        print(chunk)
    ```

여러 스트리밍 모드를 결합하여 `stream_mode` 매개변수에 리스트를 제공할 수도 있습니다:

=== "동기"

    ```python
    for chunk in graph.stream(inputs, stream_mode=["updates", "custom"]):
        print(chunk)
    ```

=== "비동기"

    ```python
    async for chunk in graph.astream(inputs, stream_mode=["updates", "custom"]):
        print(chunk)
    ```


## 설정


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


OPENAI_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 빠르게 발견하고 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축된 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 활용할 수 있습니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 읽어보세요.
    </p>
</div>


두 개의 노드로 이루어진 간단한 그래프를 정의합시다:


## 그래프 정의


In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph, START


class State(TypedDict):
    topic: str
    joke: str


def refine_topic(state: State):
    return {"topic": state["topic"] + " and cats"}


def generate_joke(state: State):
    return {"joke": f"This is a joke about {state['topic']}"}


graph = (
    StateGraph(State)
    .add_node(refine_topic)
    .add_node(generate_joke)
    .add_edge(START, "refine_topic")
    .add_edge("refine_topic", "generate_joke")
    .compile()
)


## 상태의 모든 값 스트리밍 (stream_mode="values") {#values}


이것을 사용하여 각 단계 후 **모든 값**을 상태에서 스트리밍합니다.


In [4]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    # highlight-next-line
    stream_mode="values",
):
    print(chunk)


{'topic': 'ice cream'}
{'topic': 'ice cream and cats'}
{'topic': 'ice cream and cats', 'joke': 'This is a joke about ice cream and cats'}


## 노드에서 상태 업데이트 스트리밍 (stream_mode="updates") {#updates}


각 단계 후 노드에서 반환된 **상태 업데이트**만 스트리밍하는 데 사용하세요. 스트리밍된 출력에는 노드의 이름과 업데이트가 포함됩니다.


In [5]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    # highlight-next-line
    stream_mode="updates",
):
    print(chunk)


{'refine_topic': {'topic': 'ice cream and cats'}}
{'generate_joke': {'joke': 'This is a joke about ice cream and cats'}}


## 스트림 디버그 이벤트 (stream_mode="debug") {#debug}


각 단계별로 가능한 한 많은 정보를 포함하여 **디버그 이벤트**를 스트리밍하는 데 사용합니다. 여기에는 실행될 예정인 작업에 대한 정보와 작업 실행 결과가 포함됩니다.


In [6]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    # highlight-next-line
    stream_mode="debug",
):
    print(chunk)


{'type': 'task', 'timestamp': '2025-01-28T22:06:34.789803+00:00', 'step': 1, 'payload': {'id': 'eb305d74-3460-9510-d516-beed71a63414', 'name': 'refine_topic', 'input': {'topic': 'ice cream'}, 'triggers': ['start:refine_topic']}}
{'type': 'task_result', 'timestamp': '2025-01-28T22:06:34.790013+00:00', 'step': 1, 'payload': {'id': 'eb305d74-3460-9510-d516-beed71a63414', 'name': 'refine_topic', 'error': None, 'result': [('topic', 'ice cream and cats')], 'interrupts': []}}
{'type': 'task', 'timestamp': '2025-01-28T22:06:34.790165+00:00', 'step': 2, 'payload': {'id': '74355cb8-6284-25e0-579f-430493c1bdab', 'name': 'generate_joke', 'input': {'topic': 'ice cream and cats'}, 'triggers': ['refine_topic']}}
{'type': 'task_result', 'timestamp': '2025-01-28T22:06:34.790337+00:00', 'step': 2, 'payload': {'id': '74355cb8-6284-25e0-579f-430493c1bdab', 'name': 'generate_joke', 'error': None, 'result': [('joke', 'This is a joke about ice cream and cats')], 'interrupts': []}}


## 스트림 LLM 토큰 ([stream_mode="messages"](../streaming-tokens)) {#messages}


LLM 호출을 포함하도록 위의 예제를 수정합시다. 이를 통해 **LLM 메시지를 토큰 단위로** 스트리밍하고 노드나 작업 내에서 LLM 호출을 위한 메타데이터를 함께 사용할 수 있습니다.


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


def generate_joke(state: State):
    # highlight-next-line
    llm_response = llm.invoke(
        # highlight-next-line
        [
            # highlight-next-line
            {"role": "user", "content": f"Generate a joke about {state['topic']}"}
            # highlight-next-line
        ]
        # highlight-next-line
    )
    return {"joke": llm_response.content}


graph = (
    StateGraph(State)
    .add_node(refine_topic)
    .add_node(generate_joke)
    .add_edge(START, "refine_topic")
    .add_edge("refine_topic", "generate_joke")
    .compile()
)


In [8]:
for message_chunk, metadata in graph.stream(
    {"topic": "ice cream"},
    # highlight-next-line
    stream_mode="messages",
):
    if message_chunk.content:
        print(message_chunk.content, end="|", flush=True)


Why| did| the| cat| sit| on| the| ice| cream| cone|?

|Because| it| wanted| to| be| a| "|p|urr|-f|ect|"| scoop|!| 🍦|🐱|

In [9]:
metadata


{'langgraph_step': 2,
 'langgraph_node': 'generate_joke',
 'langgraph_triggers': ['refine_topic'],
 'langgraph_path': ('__pregel_pull', 'generate_joke'),
 'langgraph_checkpoint_ns': 'generate_joke:568879bc-8800-2b0d-a5b5-059526a4bebf',
 'checkpoint_ns': 'generate_joke:568879bc-8800-2b0d-a5b5-059526a4bebf',
 'ls_provider': 'openai',
 'ls_model_name': 'gpt-4o-mini',
 'ls_model_type': 'chat',
 'ls_temperature': 0.7}

## 사용자 지정 데이터 스트리밍 (stream_mode="custom") {#custom}


[`StreamWriter`][langgraph.types.StreamWriter]를 사용하여 노드 내부에서 커스텀 데이터를 스트리밍하십시오.


In [10]:
from langgraph.types import StreamWriter


# highlight-next-line
def generate_joke(state: State, writer: StreamWriter):
    # highlight-next-line
    writer({"custom_key": "Writing custom data while generating a joke"})
    return {"joke": f"This is a joke about {state['topic']}"}


graph = (
    StateGraph(State)
    .add_node(refine_topic)
    .add_node(generate_joke)
    .add_edge(START, "refine_topic")
    .add_edge("refine_topic", "generate_joke")
    .compile()
)


In [11]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    # highlight-next-line
    stream_mode="custom",
):
    print(chunk)


{'custom_key': 'Writing custom data while generating a joke'}


## 여러 스트리밍 모드 구성하기 {#multiple}


여러 스트리밍 모드를 결합하는 데 사용하십시오. 출력은 튜플 `(스트림_모드, 스트리밍_출력)`로 스트리밍됩니다.


In [12]:
for stream_mode, chunk in graph.stream(
    {"topic": "ice cream"},
    # highlight-next-line
    stream_mode=["updates", "custom"],
):
    print(f"Stream mode: {stream_mode}")
    print(chunk)
    print("\n")


Stream mode: updates
{'refine_topic': {'topic': 'ice cream and cats'}}


Stream mode: custom
{'custom_key': 'Writing custom data while generating a joke'}


Stream mode: updates
{'generate_joke': {'joke': 'This is a joke about ice cream and cats'}}


